## Цель ноутбука

Познакомиться с подходом transfer learning для построения и обучения объёмных нейронных сетей. Воспользовавшись предобученной моделью [ResNet](https://pytorch.org/hub/pytorch_vision_resnet/), мы построим классификатор изображений с кошками и собаками на датасете [KaggleCatsAndDogs](https://www.microsoft.com/en-us/download/details.aspx?id=54765).

Модель ResNet построена на свёртках. Эта матричная операция занимает много времени, будучи выполняемой на CPU. Архитектура CUDA позволяет существенно ускорить вычисление матричных операций благодаря параллельным вычислениям и использованию графических процессоров. Именно поэтому с этим ноутбуком лучше работать в Google Colab, так как он даёт возможность работы с видеокартой.

### 1. Готовимся к работе.
- Подключаем Google Drive.
- Устанавливаем и импортируем необходимые библиотеки.
- Извлекаем данные из архива.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install torchinfo
!unzip "/content/drive/MyDrive/Skillbox/ML Advanced/kagglecatsanddogs.zip"

In [ ]:
import os
from glob import glob
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import ToTensor, Compose, Resize, Normalize, ToPILImage
from PIL import Image
from torchinfo import summary
from torchvision import models

### 2. Готовим датасет.
- Описываем класс кастомного датасета, наследуясь от `torch.utils.data.Dataset`
- Загружаем датасет и делим его на тренировочную и валидационную выборки.
- Оборачиваем датасеты в DataLoader.

Обратите внимание на то, что мы нормируем и центрируем наши данные не на (0.5, 0.5), а на заранее вычисленные среднее и корень из дисперсии изображений из датасета ImageNET. Так мы приближаем наши данные к тому, с чем училась работать модель ResNet. По этой же причине мы приводим все данные к размеру 224 × 224 пикселя.

In [ ]:
class CatsDogsDataset(Dataset):
    def __init__(self, datapath, transform=None):
        super(CatsDogsDataset, self).__init__()
        self.paths = []
        self.labels = []

        for y, cls in enumerate(['Cat', 'Dog']):
            p_tmp = glob(os.path.join(datapath, 'PetImages', cls, '*.jpg'))
            l_tmp = [y] * len(p_tmp)
            self.paths.extend(p_tmp)
            self.labels.extend(l_tmp)

        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img = Image.open(self.paths[idx])
        label = self.labels[idx]

        if self.transform is not None:
            img = self.transform(img)

        return img, label

In [ ]:
dataset = CatsDogsDataset('/content/kagglecatsanddogs')
for i in range(10):
    display(dataset[i][0])
    display(dataset[-i][0])

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
transform = Compose([
    ToTensor(),
    Resize((224, 224)),
    Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset = CatsDogsDataset('/content/kagglecatsanddogs', transform=transform)

random_generator = torch.Generator().manual_seed(42)
train_dataset, val_dataset = random_split(dataset, [0.8, 0.2], generator=random_generator)
print(len(train_dataset), len(val_dataset))

train_loader = DataLoader(train_dataset, batch_size=200, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1000)

### 3. Собираем модель.
- Скачиваем веса обученной модели ResNet, например resnet-50.
- Замораживаем параметры модели путём отключения для них процесса вычисления градиентов и обновления весов.
- Меняем последний слой под задачу бинарной классификации.

Для задачи бинарной классификации мы выберем бинарную кросс-энтропию в качестве лосса — [BCELoss](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html). В отличие от CrossEntropyLoss, BCELoss ожидает на вход именно вероятность принадлежности семпла к целевому классу. Именно поэтому нам обязательно нужно применить сигмоиду к выходу линейного слоя.

In [ ]:
resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
summary(resnet, input_size=(1, 3, 224, 224))

In [ ]:
for param in resnet.parameters():
    param.requires_grad = False

fc_inputs = resnet.fc.in_features # 2048
resnet.fc = nn.Sequential(
    nn.Linear(fc_inputs, 1),
    nn.Sigmoid()
)

### 4. Обучаем модель и сохраняем веса.
Пишем функции `train()` и `validate()`. Здесь добавится новый аргумент функций — `device`, так как мы хотим работать с моделью на GPU, но должны иметь возможность запустить её также и на CPU.

Когда вы запускаете долгое обучение модели, обязательно предусмотрите периодическое сохранение весов. В нашем случае обучение может занять до получаса, и допустимо сохранить веса только в конце. При этом важно, чтобы ячейка с сохранением была запущена сразу же с обучением и ждала своей очереди на выполнение. Иначе можно потерять все результаты, так как Google Colab заберёт у вас видеокарту и отключит ядро в случае бездействия. По этой же причине нужно сохранять веса именно на Google Drive.

In [ ]:
def train(model, optimizer, loss_f, train_loader, val_loader, n_epoch, val_fre, device):
    model = model.to(device)
    model.train()
    for epoch in range(n_epoch):
        loss_sum = 0
        print(f'Epoch: {epoch}')
        for step, (data, target) in enumerate(train_loader):
            target = target.to(torch.float32)
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data).squeeze(1)
            loss = loss_f(output, target)
            loss.backward()
            optimizer.step()

            loss_sum += loss.item()

            if step % 10 == 0:
                print(f'Iter: {step} \tLoss: {loss.item()}')

        print(f'Mean Train Loss: {loss_sum / (step + 1):.6f}', end='\n\n')

        if epoch % val_fre == 0:
            validate(model, val_loader, device)

def validate(model, val_loader, device):
    model = model.to(device)
    model.eval()
    loss_sum = 0
    correct = 0
    for step, (data, target) in enumerate(val_loader):
        target = target.to(torch.float32)
        data, target = data.to(device), target.to(device)
        with torch.no_grad():
            output = model(data).squeeze(1)
            loss = loss_f(output, target)
        loss_sum += loss.item()
        pred = torch.round(output)
        correct += pred.eq(target.view_as(pred)).sum().item()
    acc = correct / len(val_loader.dataset)
    print(f'Val Loss: {loss_sum / (step + 1):.6f} \tAccuracy: {acc}')
    model.train()

In [ ]:
loss_f = nn.BCELoss()
optimizer = torch.optim.SGD(resnet.parameters(), lr=1e-3)

n_epoch = 10
val_fre = 1

device = 'cuda'

# train(resnet, optimizer, loss_f, train_loader, val_loader, n_epoch, val_fre, device)
# torch.save(resnet.fc.state_dict(), '/content/drive/MyDrive/Skillbox/ML Advanced/resnet_fc.pth')
resnet.fc.load_state_dict(torch.load('/content/drive/MyDrive/Skillbox/ML Advanced/resnet_fc.pth'))

validate(resnet, val_loader, device)